<a href="https://colab.research.google.com/github/tiagopessoalima/TATI/blob/main/Semana_11_(TATI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Banco de Dados e SQL**

Este notebook tem como objetivo apresentar os fundamentos da manipulação e consulta de dados utilizando **SQL** em conjunto com a biblioteca **pandas**. Serão abordados os princípios básicos de bancos de dados relacionais, a sintaxe SQL para operações de consulta (`SELECT`, `WHERE`, `JOIN`, entre outras) e, principalmente, a integração desses recursos ao pandas, permitindo a extração, transformação e análise de dados de forma eficiente e escalável.

## **O que é um Banco de Dados e SQL?**

* **Banco de Dados Relacional:** Sistema estruturado para armazenamento, organização e gerenciamento de dados, baseado no modelo relacional. Representa os dados em tabelas (relações) compostas por linhas (tuplas) e colunas (atributos), permitindo integridade, consistência e acesso eficiente.

* **SQL (Structured Query Language):** Linguagem declarativa padrão para interação com bancos de dados relacionais. Permite realizar operações como consulta (`SELECT`), inserção (`INSERT`), atualização (`UPDATE`) e remoção (`DELETE`) de dados, além de manipulação de estruturas e controle de permissões.

* **Integração Pandas + SQL:** A combinação de SQL com a biblioteca pandas oferece uma abordagem robusta para análise de dados. O SQL é responsável pela extração eficiente dos dados diretamente da base, aplicando filtros, junções e agregações no servidor de dados. O pandas complementa essa etapa ao possibilitar manipulações avançadas, transformações, análises estatísticas e visualizações diretamente na memória, maximizando a produtividade na análise de dados.


## **Preparando o Ambiente**

O **SQLite** é uma solução leve, que não requer a configuração de um servidor dedicado, armazenando os dados em um único arquivo local ou diretamente na memória. Por essas características, é utilizado em ambientes de desenvolvimento, prototipagem, testes e aplicações embarcadas. Inicialmente, realizaremos a importação das bibliotecas necessárias e criaremos uma instância de banco de dados **em memória** (`:memory:`), que reside na RAM e não persiste após o encerramento da sessão.

In [ ]:
import pandas as pd
import sqlite3

# Criando uma conexão com um banco de dados SQLite em memória
# Isso significa que o banco de dados existirá apenas enquanto o script estiver rodando
conn = sqlite3.connect(':memory:')

# Para confirmar que a conexão foi criada
print("Conexão com o banco de dados SQLite criada com sucesso!")

Conexão com o banco de dados SQLite criada com sucesso!


Agora, vamos criar alguns DataFrames com o pandas e carregá-los como tabelas no nosso banco de dados.

In [ ]:
# Criando DataFrame de funcionários
df_funcionarios = pd.DataFrame({
    'id_funcionario': [1, 2, 3, 4, 5, 6, 7],
    'nome': ['Ana', 'Bruno', 'Carla', 'Daniel', 'Eva', 'Fábio', 'Gisele'],
    'id_departamento': [1, 2, 1, 3, 2, 1, 3],
    'salario': [7000, 8200, 6500, 9500, 11000, 5800, 12000]
})

# Criando DataFrame de departamentos
df_departamentos = pd.DataFrame({
    'id_departamento': [1, 2, 3],
    'nome_departamento': ['TI', 'Marketing', 'Vendas']
})

# Usando a função to_sql para enviar os dados dos DataFrames para o banco de dados
df_funcionarios.to_sql('funcionarios', conn, index=False, if_exists='replace')
df_departamentos.to_sql('departamentos', conn, index=False, if_exists='replace')

print("Tabelas 'funcionarios' e 'departamentos' criadas no banco de dados.")

Tabelas 'funcionarios' e 'departamentos' criadas no banco de dados.


## **Consultas Básicas com SQL**

A principal forma de ler dados de um banco SQL com pandas é usando a função `pd.read_sql_query()`.

### **SELECT - Selecionando Colunas**

Para selecionar todas as colunas de uma tabela, usamos *.

In [ ]:
# Consulta SQL para selecionar tudo da tabela de funcionários
query_todos_funcionarios = "SELECT * FROM funcionarios;"

# Executando a consulta e carregando o resultado em um DataFrame
df_resultado = pd.read_sql_query(query_todos_funcionarios, conn)

print("Todos os funcionários:")
print(df_resultado)

Todos os funcionários:
   id_funcionario    nome  id_departamento  salario
0               1     Ana                1     7000
1               2   Bruno                2     8200
2               3   Carla                1     6500
3               4  Daniel                3     9500
4               5     Eva                2    11000
5               6   Fábio                1     5800
6               7  Gisele                3    12000


Para selecionar colunas específicas, basta listá-las.

In [ ]:
# Selecionando apenas nome e salário
query_nome_salario = "SELECT nome, salario FROM funcionarios;"

df_resultado = pd.read_sql_query(query_nome_salario, conn)

print("Nome e salário dos funcionários:")
print(df_resultado)

Nome e salário dos funcionários:
     nome  salario
0     Ana     7000
1   Bruno     8200
2   Carla     6500
3  Daniel     9500
4     Eva    11000
5   Fábio     5800
6  Gisele    12000


## **WHERE - Filtrando Dados**

A cláusula `WHERE` é usada para filtrar registros com base em uma condição.

In [ ]:
# Encontrar funcionários com salário maior que 8000
query_salario_alto = "SELECT * FROM funcionarios WHERE salario > 8000;"

df_resultado = pd.read_sql_query(query_salario_alto, conn)

print("Funcionários com salário acima de 8000:")
print(df_resultado)

Funcionários com salário acima de 8000:
   id_funcionario    nome  id_departamento  salario
0               2   Bruno                2     8200
1               4  Daniel                3     9500
2               5     Eva                2    11000
3               7  Gisele                3    12000


## **LIMIT - Limitando o Número de Resultados**

Se a tabela for muito grande, podemos querer ver apenas as primeiras linhas.

In [ ]:
# Selecionar os 3 primeiros funcionários
query_limit = "SELECT * FROM funcionarios LIMIT 3;"

df_resultado = pd.read_sql_query(query_limit, conn)

print("Os 3 primeiros funcionários da tabela:")
print(df_resultado)

Os 3 primeiros funcionários da tabela:
   id_funcionario   nome  id_departamento  salario
0               1    Ana                1     7000
1               2  Bruno                2     8200
2               3  Carla                1     6500


## **Consultas Avançadas com SQL**

Agora vamos explorar algumas funcionalidades mais poderosas do SQL.

### **JOIN - Combinando Tabelas**

`JOIN` é usado para combinar linhas de duas ou more tabelas com base em uma coluna relacionada. Vamos juntar funcionarios com departamentos para ver o nome do departamento de cada funcionário.

In [ ]:
# Usando INNER JOIN para combinar as tabelas
query_join = """
SELECT
    f.nome,
    f.salario,
    d.nome_departamento
FROM
    funcionarios AS f
INNER JOIN
    departamentos AS d ON f.id_departamento = d.id_departamento;
"""

df_join = pd.read_sql_query(query_join, conn)

print("Funcionários com seus respectivos departamentos:")
print(df_join)

> **INNER JOIN:** Retorna apenas os registros que têm valores correspondentes em ambas as tabelas.

> **LEFT JOIN:** Retorna todos os registros da tabela da esquerda (funcionarios) e os registros correspondentes da tabela da direita (departamentos). Se não houver correspondência, o resultado é NULL no lado direito.

### **GROUP BY - Agrupando Dados**

A cláusula `GROUP BY` é utilizada para agrupar registros que compartilham o mesmo valor em um ou mais atributos, produzindo uma tupla de saída para cada grupo distinto. Essa operação é essencial em consultas que envolvem **funções de agregação** — como `COUNT()`, `AVG()`, `SUM()`, `MIN()`, `MAX()` — permitindo calcular métricas resumidas por categorias específicas. No exemplo a seguir, aplicaremos a cláusula `GROUP BY` para calcular a **média salarial por departamento**.


In [ ]:
# Calcular salário médio e número de funcionários por departamento
query_group_by = """
SELECT
    d.nome_departamento,
    COUNT(f.id_funcionario) AS numero_de_funcionarios,
    AVG(f.salario) AS salario_medio
FROM
    funcionarios AS f
INNER JOIN
    departamentos AS d ON f.id_departamento = d.id_departamento
GROUP BY
    d.nome_departamento;
"""

df_agregado = pd.read_sql_query(query_group_by, conn)

print("Análise de salários por departamento:")
print(df_agregado)

Análise de salários por departamento:
  nome_departamento  numero_de_funcionarios  salario_medio
0         Marketing                       2    9600.000000
1                TI                       3    6433.333333
2            Vendas                       2   10750.000000


## **ORDER BY - Ordenando os Resultados**

Podemos ordenar os resultados de uma consulta em ordem ascendente (ASC) ou descendente (DESC).

In [ ]:
# Listar funcionários do mais bem pago para o menos bem pago
query_order_by = "SELECT nome, salario FROM funcionarios ORDER BY salario DESC;"

df_ordenado = pd.read_sql_query(query_order_by, conn)

print("Funcionários ordenados por salário (do maior para o menor):")
print(df_ordenado)

Funcionários ordenados por salário (do maior para o menor):
     nome  salario
0  Gisele    12000
1     Eva    11000
2  Daniel     9500
3   Bruno     8200
4     Ana     7000
5   Carla     6500
6   Fábio     5800


## **HAVING - Filtrando Grupos**

A cláusula **`WHERE`** aplica filtros sobre as tuplas da tabela **antes da execução da agregação**, restringindo o conjunto de dados que será processado. Por outro lado, a cláusula **`HAVING`** atua **após a etapa de agrupamento**, permitindo impor condições sobre os grupos resultantes de um `GROUP BY`. No exemplo a seguir, utilizaremos `HAVING` para restringir a saída aos departamentos cuja **média salarial (`AVG`) excede 9000**, ou seja, o filtro é aplicado sobre o resultado da função de agregação calculada por grupo.


In [ ]:
# Usando HAVING para filtrar os resultados do GROUP BY
query_having = """
SELECT
    d.nome_departamento,
    AVG(f.salario) AS salario_medio
FROM
    funcionarios AS f
INNER JOIN
    departamentos AS d ON f.id_departamento = d.id_departamento
GROUP BY
    d.nome_departamento
HAVING
    AVG(f.salario) > 9000;
"""

df_filtrado_grupo = pd.read_sql_query(query_having, conn)

print("Departamentos com salário médio acima de 9000:")
print(df_filtrado_grupo)

Departamentos com salário médio acima de 9000:
  nome_departamento  salario_medio
0         Marketing         9600.0
1            Vendas        10750.0


A consulta a seguir demonstra o uso da cláusula **`WHERE`** para aplicar filtros no nível das tuplas individuais, antes da etapa de agrupamento. Diferentemente da cláusula **`HAVING`**, que filtra grupos pós-agregação, o **`WHERE`** restringe o conjunto de dados brutos, selecionando somente os registros cujo atributo `salario` excede 9000. Como consequência, a função agregada `AVG` calculará a média apenas sobre esse subconjunto pré-filtrado, o que pode resultar em valores distintos em comparação à filtragem realizada após o agrupamento.

In [ ]:
query_where = """
SELECT
    d.nome_departamento,
    AVG(f.salario) AS salario_medio
FROM
    funcionarios AS f
INNER JOIN
    departamentos AS d ON f.id_departamento = d.id_departamento
WHERE
    f.salario > 9000
GROUP BY
    d.nome_departamento;
"""

df_filtrado_where = pd.read_sql_query(query_where, conn)

print("Salário médio considerando apenas funcionários com salário > 9000:")
print(df_filtrado_where)


Salário médio considerando apenas funcionários com salário > 9000:
  nome_departamento  salario_medio
0         Marketing        11000.0
1            Vendas        10750.0


## **Conclusão**

Nesta aula, demonstramos como a biblioteca **pandas** pode ser utilizada como uma interface eficiente para interação com bancos de dados relacionais via SQL.
Abordamos a criação de bancos de dados e tabelas a partir de estruturas **DataFrame**, possibilitando a persistência de dados. Executamos consultas SQL tanto básicas (`SELECT`, `WHERE`) quanto avançadas (`JOIN`, `GROUP BY`, `ORDER BY`), ilustrando diferentes níveis de manipulação e extração de dados. Os resultados das consultas foram carregados diretamente em DataFrames, facilitando a análise exploratória e manipulação posterior dos dados. Essa abordagem integra a performance e expressividade do SQL para extração precisa de dados com a versatilidade do pandas para análises estatísticas, transformações e visualizações detalhadas. Por fim, é recomendada a correta finalização da sessão com o fechamento da conexão ao banco de dados:

In [ ]:
conn.close()
print("Conexão com o banco de dados fechada.")

Conexão com o banco de dados fechada.


## **Quiz: Agrupando e Filtrando com SQL**

Teste seus conhecimentos e pratique os conceitos abordados nesta aula!

https://g.co/gemini/share/7cb1c52af389










## **Exercícios Práticos**

**Exercício 1: Funcionários de Marketing**

Selecione o nome e o salário de todos os funcionários que trabalham no departamento de 'Marketing'.

**Exercício 2: O Funcionário Mais Antigo (por ID)**

Encontre o nome do funcionário com o menor id_funcionario, assumindo que IDs menores são mais antigos.

**Exercício 3: Departamentos com Poucos Funcionários**

Liste os nomes dos departamentos que têm 2 ou menos funcionários.